In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#VARIABLES
dataset=r"../data/ev_charging_dataset_Final(in).csv"

In [ ]:
df = pd.read_csv(dataset)
df.head(5).display()

df.describe().display()